In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import NearestNDInterpolator
import glob
import os
import matplotlib.pyplot as plt
import scipy.stats as st

# Historical calcs using CRU

In [ ]:
pet_ds = xr.open_dataset('./data/cru_ts4.08.1901.2023.pet.dat.nc')

In [ ]:
pet_ds = pet_ds.sel(time=slice(np.datetime64('2021-01-01'), np.datetime64('2021-12-31'))).mean(dim='time')
# pet_ds = pet_ds.sel(time=slice(np.datetime64('1940-01-01'), np.datetime64('1949-12-31'))).mean(dim='time')

In [ ]:
res_df = pd.read_csv('../remove_bad_water/out/sentinel_2021_v6_wgs84_combined_merged.csv')
res_df = res_df.loc[res_df['hydropoly_max']<100]
res_df['area_ha'] = res_df['area']*100/10000 # HA
res_df['area_km'] = res_df['area']*100/(1000*1000) # km2
res_df = res_df.loc[res_df['area_ha']<100]
res_df['area_m'] = res_df['area']*100

In [ ]:
res_df['area_km'].sum()

In [ ]:
res_df.shape

In [ ]:
xr_indexer = {
    'lon':xr.DataArray(res_df['longitude'].values, dims=['res']),
    'lat':xr.DataArray(res_df['latitude'].values, dims=['res'])
}

In [ ]:
res_pets = pet_ds.sel(xr_indexer, method='nearest')
res_pets = res_pets.assign(area_m=('res', res_df['area_m'].values))

In [ ]:
res_month_evap = res_pets['pet']*res_pets['area_m']
res_month_evap = res_month_evap/1000 # Assuming PET is in mm
evap_value = (res_month_evap*365).sum()*1000 # 365 days per year, 1000 converts from cubic meters to liters

In [ ]:
print(evap_value)

# Future Calcs

Using: https://www.nature.com/articles/s41597-023-02290-0


In [ ]:
def interp(da):
    indices = np.where(np.isfinite(da))
    interp = NearestNDInterpolator(np.transpose(indices), da.data[indices])
    da[...] = interp(*np.indices(da.shape))
    return da


def calc_total_evap(res_pets):
    res_daily_evap = res_pets['PET']*res_pets['area_m'] / 1000 # Convert PET from mm to m
    evap_value = (res_daily_evap*365).sum() * 1000 # Convert to annual cubic liters (not m^3)
    return evap_value

def full_pet_calcs(res_df, pet_ncdf_path):
    pet_ds = xr.open_dataset(pet_ncdf_path)
    xr_indexer = {
        'lon':xr.DataArray(res_df['longitude'].values, dims=['res']),
        'lat':xr.DataArray(res_df['latitude'].values, dims=['res'])
    }
    pet_ds_last10 = pet_ds.sel(time=slice('2090-01-01', '2100-01-01')).mean(dim='time')
    pet_ds_first10 = pet_ds.sel(time=slice('2015-01-01', '2025-01-01')).mean(dim='time')
    pet_ds_first10['PET'] = interp(pet_ds_first10['PET'])
    pet_ds_last10['PET'] = interp(pet_ds_last10['PET'])
    res_pets_last10 = pet_ds_last10.sel(xr_indexer, method='nearest')
    res_pets_last10 = res_pets_last10.assign(area_m=('res', res_df['area_m'].values))
    res_pets_first10 = pet_ds_first10.sel(xr_indexer, method='nearest')
    res_pets_first10 = res_pets_first10.assign(area_m=('res', res_df['area_m'].values))
    evap_first10 = calc_total_evap(res_pets_first10)
    evap_last10 = calc_total_evap(res_pets_last10)
    return evap_first10, evap_last10

def annual_pet_calcs(res_df, pet_ncdf_path):
    pet_ds = xr.open_dataset(pet_ncdf_path)
    xr_indexer = {
        'lon':xr.DataArray(res_df['longitude'].values, dims=['res']),
        'lat':xr.DataArray(res_df['latitude'].values, dims=['res'])
    }

    evap_list = []

    for y in range(2021, 2101):
        pet_ds_year = pet_ds.sel(time=slice('{}-01-01'.format(y), '{}-12-30'.format(y))).mean(dim='time')
        pet_ds_year['PET'] = interp(pet_ds_year['PET'])
        res_pets_year = pet_ds_year.sel(xr_indexer, method='nearest')
        res_pets_year = res_pets_year.assign(area_m=('res', res_df['area_m'].values))
        evap_year = calc_total_evap(res_pets_year)
        evap_list.append(evap_year.values)
    return evap_list

##  First/last evap

In [ ]:
first_last_csv = './data/sentinel_v6_2021_allmodels_cmip6.csv'
if not os.path.isfile(first_last_csv):
    # Do All and write out results
    all_models = glob.glob('./data/cmip6/*.nc')
    all_dicts = []
    for mod_nc in all_models:
        mod_name = os.path.basename(mod_nc)[:-3]
        print(mod_name)
        baseline, final = full_pet_calcs(res_df, mod_nc)
        out_dict = {
            'model':mod_name,
            'baseline_evap': baseline.values,
            'final_evap': final.values
        }
        all_dicts.append(out_dict)
    full_df = pd.DataFrame(all_dicts)
    full_df.to_csv('./data/sentinel_v6_2021_allmodels_cmip6.csv', index=False)

In [ ]:
full_df = pd.read_csv(first_last_csv)
full_df['scenario'] = None
full_df.loc[full_df['model'].str.contains('ssp126'), 'scenario'] = 'ssp126'
full_df.loc[full_df['model'].str.contains('ssp245'), 'scenario'] = 'ssp245'
full_df.loc[full_df['model'].str.contains('ssp370'), 'scenario'] = 'ssp370'
full_df.loc[full_df['model'].str.contains('ssp585'), 'scenario'] = 'ssp585'
full_df['ratio'] = (full_df['final_evap'] / full_df['baseline_evap'])

In [ ]:
print(full_df.groupby('scenario').aggregate(['mean','std', 'max', 'min', 'count'])['ratio'])
print(full_df.groupby('scenario').aggregate(['mean','std', 'max', 'min', 'count'])['ratio']['mean']*11700 - 11700)

### Annual evap

In [ ]:
annual_csv = './data/sentinel_v6_2021_allmodels_annual_cmip6.csv'
if not os.path.isfile(annual_csv):
    out_dict = {
        'year':np.arange(2021,2101)
    }
    all_models = glob.glob('./data/cmip6/*.nc')
    for mod_nc in all_models:
        mod_name = os.path.basename(mod_nc)[:-3]
        print(mod_name)
        annual_evap = annual_pet_calcs(res_df, mod_nc)
        out_dict[mod_name] = annual_evap

    pd.DataFrame(out_dict).set_index('year').to_csv('./data/sentinel_v6_2021_allmodels_annual_cmip6.csv')

In [ ]:
annual_df = pd.read_csv('./data/sentinel_v6_2021_allmodels_annual_cmip6.csv').set_index('year')
annual_df.index = annual_df.index.astype(int)

In [ ]:
# Scale to beginning
annual_df_scaler = evap_value.values/annual_df.loc[2021]
annual_df = annual_df*annual_df_scaler
annual_df = annual_df/1000000000000 # Scale to km cubed
# Smoothing, if wanted
if True:
    for col in annual_df.columns:
        annual_df.loc[:, col] = np.convolve(annual_df[col], np.ones(3)/3, mode='same')
    annual_df.loc[2021] = annual_df.loc[2021]*1.5
    annual_df.loc[2100] = annual_df.loc[2100]*1.5

In [ ]:
annual_df_rotate = annual_df.T
annual_df_rotate['scenario'] = None
annual_df_rotate.loc[annual_df_rotate.index.str.contains('ssp126'), 'scenario'] = 'SSP1-2.6'
annual_df_rotate.loc[annual_df_rotate.index.str.contains('ssp245'), 'scenario'] = 'SSP2-4.5'
annual_df_rotate.loc[annual_df_rotate.index.str.contains('ssp370'), 'scenario'] = 'SSP3-7.0'
annual_df_rotate.loc[annual_df_rotate.index.str.contains('ssp585'), 'scenario'] = 'SSP5-8.5'

In [ ]:
def calc_interval_low(a):
    return st.t.interval(0.8, len(a)-1, loc=np.mean(a), scale=st.sem(a))[0]
def calc_interval_high(a):
    return st.t.interval(0.8, len(a)-1, loc=np.mean(a), scale=st.sem(a))[1]

In [ ]:

annual_mean = annual_df_rotate.groupby('scenario').mean().T
annual_std = annual_df_rotate.groupby('scenario').std().T
annual_min = annual_df_rotate.groupby('scenario').min().T
annual_max = annual_df_rotate.groupby('scenario').max().T
# Compute CI
annual_low = annual_min.copy()
annual_high = annual_max.copy()
for scenario in annual_mean.columns:
    annual_low.loc[:,scenario] = annual_df_rotate.loc[annual_df_rotate['scenario']==scenario].drop(columns=['scenario']).apply(calc_interval_low)
    annual_high.loc[:,scenario] = annual_df_rotate.loc[annual_df_rotate['scenario']==scenario].drop(columns=['scenario']).apply(calc_interval_high)

In [ ]:
color_list = ['#6a0572','#f77f00', '#168AAD','forestgreen']
linestyle_list = [':','--','-.','-']
fig, ax = plt.subplots(figsize=(3.75,3.75))
for i, scenario in enumerate(annual_mean.columns[::-1]):
    lw=1.4
    if i == 0:
        lw=1.7
    annual_mean[scenario].plot(color=color_list[i], style=linestyle_list[i], ax=ax, lw=lw)
    ax.fill_between(annual_df.index,
                    annual_low[scenario],
                    annual_high[scenario],
                    color=color_list[i],
                    alpha=0.15, lw=0.9)
ax.axhline(y=11.74,color='black', lw=1.0, label='Current', linestyle=(0, (5,8)))
ax.set_xlim(2020, 2100)
ax.set_xticks([2020, 2040, 2060, 2080, 2100])
ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Evaporation ($km^3y^{-1}$)')
# plt.savefig('/home/ksolvik/research/reservoirs/figs/ch0/evaporation.jpg', dpi=300,
#             pil_kwargs={'quality':95},
#             bbox_inches='tight')
